In [ ]:
pip install requests newspaper3k lxml_html_clean bs4 fake_useragent python-docx


In [ ]:
# The guardian
# Premium times
# The Nation


[]

In [13]:
import requests
from newspaper import Article, Config

def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config = config)

    try:
        page.download()
        page.parse()
        return page.text, page.authors, page.publish_date
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'

full_content('https://thenationonlineng.net/bbnaija-s10-19-housemates-face-possible-eviction/')

('BBNaija S10: 19 housemates face possible eviction\n\nNineteen Big Brother Naija housemates have been nominated for possible eviction in week six.\n\nThe current Head of House, Faith, his immunity-savee Zita and ‘Immunity’ card holder, Mensan are the only three housemates spared from the chopping block.\n\nThe housemates were previously tasked with searching for envelopes in the house, with strict instructions to maintain confidentiality.\n\nRead Also: Simi set to thrill fans in Lagos concert\n\nThe season, which began with 29 housemates vying for the top prize, has been filled with excitement and tension.\n\nThe winner of this season will take home a whopping N150 million grand prize, the largest prize in the show’s history.\n\nWith the stakes high, housemates are expected to form alliances, strategise, and play the game to stay in the house.\n\nThe fate of the nominated housemates now rests in the hands of the viewers.',
 ['Yewande Fasan',
  '.Wp-Block-Co-Authors-Plus-Coauthors.Is-L

In [1]:
# Read file, replace \n with comma, make each line a string
with open('/workspace/testing/thenationonlineng.net_rape_articles.txt', 'r') as file:
    content = file.read()

# Replace newlines with commas and split into strings
modified_content = content.split('\\n')
modified_content

# print(f"Found {len(strings)} strings")
# print("First 5 strings:")
# for i, string in enumerate(strings[:5], 1):
#     print(f"{i}. '{string}'")

# # If you want them as a comma-separated string:
# comma_separated = ', '.join(strings)
# print(f"\nComma-separated: {comma_separated[:200]}...")

# If you want to save the comma-separated version:
# with open('output.txt', 'w') as file:
#     file.write(comma_separated)

['https://thenationonlineng.net/22-year-old-man-nabbed-over-rape-of-minor-in-adamawa/',
 'https://thenationonlineng.net/alleged-rape-court-discharges-acquits-unibens-lecturer-ekundayo-three-years-after/',
 'https://thenationonlineng.net/appeal-court-acquits-pharmacist-jailed-for-life-over-rape/',
 'https://thenationonlineng.net/appeal-court-reserves-judgment-in-doctors-rape-conviction-appeal/',
 'https://thenationonlineng.net/appeal-court-upholds-icpcs-power-to-probe-rape-allegation-against-prof-ndifon/',
 'https://thenationonlineng.net/author/udeh-onyebuchi/',
 'https://thenationonlineng.net/bamise-lagos-brt-driver-remanded-over-alleged-rape-murder/',
 'https://thenationonlineng.net/bamise-murder-doctor-says-arrested-brt-driver-attempted-to-rape-her-inside-bus/',
 'https://thenationonlineng.net/bandits-gang-rape-expectant-mom-to-death-in-niger/',
 'https://thenationonlineng.net/bangladesh-sentences-five-men-to-death-for-gang-rape/',
 'https://thenationonlineng.net/behind-the-glitter-a

In [3]:
import requests
from newspaper import Article, Config
from docx import Document
import re
from datetime import datetime
import os

def clean_filename(filename):
    """Clean filename by removing invalid characters"""
    # Remove or replace invalid characters for filenames
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)
    filename = re.sub(r'[^\w\s-]', '', filename)
    filename = re.sub(r'[-\s]+', '-', filename)
    return filename.strip('-')

def get_article_and_save_doc(url):
    """
    Get article content, title, date and save as Word doc
    Filename format: "Title - Date.docx"
    """
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config=config)

    try:
        print(f"Downloading article from: {url}")
        page.download()
        page.parse()
        
        # Get article details
        title = page.title if page.title else "Untitled Article"
        content = page.text if page.text else "No content retrieved"
        publish_date = page.publish_date
        authors = page.authors if page.authors else ["Unknown Author"]
        
        # Format date for filename
        if publish_date:
            date_str = publish_date.strftime("%Y-%m-%d")
        else:
            date_str = datetime.now().strftime("%Y-%m-%d")
        
        # Create filename: "Title - Date.docx"
        filename = f"{clean_filename(title)} - {date_str}.docx"
        
        # Limit filename length to avoid issues
        if len(filename) > 200:
            short_title = clean_filename(title)[:150]
            filename = f"{short_title} - {date_str}.docx"
        
        # Create Word document
        doc = Document()
        
        # Add title as heading
        doc.add_heading(title, 0)
        
        # Add metadata
        doc.add_paragraph(f"Date: {date_str}")
        doc.add_paragraph(f"Authors: {', '.join(authors)}")
        doc.add_paragraph(f"Source: {url}")
        doc.add_paragraph("")  # Empty line
        
        # Add content
        # Split content into paragraphs and add each one
        paragraphs = content.split('\\n\\n')
        for paragraph in paragraphs:
            if paragraph.strip():
                doc.add_paragraph(paragraph.strip())
        
        # Save document
        folder_path = "/workspace/testing/doc_files" # Example for Windows
        # Or: folder_path = "/home/youruser/MyWordDocuments" # Example for Linux/macOS
        

        # Ensure the folder exists (create it if it doesn't)
        os.makedirs(folder_path, exist_ok=True)

        # Construct the full path
        full_path = os.path.join(folder_path, filename)

        # Save the document to the specified path
        doc.save(full_path)
        
        
        print(f"✅ Article saved as: {filename}")
        print(f"Title: {title}")
        print(f"Date: {date_str}")
        print(f"Authors: {', '.join(authors)}")
        print(f"Content length: {len(content)} characters")
        
        return {
            'title': title,
            'date': publish_date,
            'authors': authors,
            'content': content,
            'filename': filename,
            'success': True
        }
        
    except Exception as e:
        print(f"❌ Error retrieving content from {url}: {e}")
        return {
            'error': str(e),
            'url': url,
            'success': False
        }

def process_multiple_urls(urls):
    """
    Process multiple URLs and save each as a separate doc file
    """
    results = []
    
    for i, url in enumerate(urls, 1):
        print(f"\\n--- Processing article {i}/{len(urls)} ---")
        result = get_article_and_save_doc(url)
        results.append(result)
        
        if result['success']:
            print(f"Success: {result['filename']}")
        else:
            print(f"Failed: {url}")
    
    # Summary
    successful = len([r for r in results if r['success']])
    print(f"\\n=== SUMMARY ===")
    print(f"Total articles processed: {len(urls)}")
    print(f"Successful: {successful}")
    print(f"Failed: {len(urls) - successful}")
    
    return results

# Example usage for single article
if __name__ == "__main__":
    # Single article
    # url = 'https://thenationonlineng.net/bbnaija-s10-19-housemates-face-possible-eviction/'
    # result = get_article_and_save_doc(url)
    
    # Multiple articles example (uncomment to use)
    # urls = [
    #     'https://thenationonlineng.net/bbnaija-s10-19-housemates-face-possible-eviction/',
    #     'https://example.com/article2',
    #     'https://example.com/article3'
    # ]
    results = process_multiple_urls(modified_content)

\n--- Processing article 1/101 ---
✅ Article saved as: 22-year-old-man-nabbed-over-rape-of-minor-in-Adamawa - 2025-02-02.docx
Title: 22-year-old man nabbed over rape of minor in Adamawa
Date: 2025-02-02
Authors: Onimisi Alao, .Wp-Block-Co-Authors-Plus-Coauthors.Is-Layout-Flow, Class, Wp-Block-Co-Authors-Plus, Display Inline, .Wp-Block-Co-Authors-Plus-Avatar, Where Img, Height Auto Max-Width, Vertical-Align Bottom .Wp-Block-Co-Authors-Plus-Coauthors.Is-Layout-Flow .Wp-Block-Co-Authors-Plus-Avatar, Vertical-Align Middle .Wp-Block-Co-Authors-Plus-Avatar Is .Alignleft .Alignright
Content length: 1453 characters
Success: 22-year-old-man-nabbed-over-rape-of-minor-in-Adamawa - 2025-02-02.docx
\n--- Processing article 2/101 ---


✅ Article saved as: Alleged-rape-court-discharges-acquits-UNIBENs-lecturer-Ekundayo-three-years-after - 2024-10-26.docx
Title: Alleged rape: court discharges, acquits UNIBEN’s lecturer, Ekundayo, three years after
Date: 2024-10-26
Authors: Bisi Olaniyi, .Wp-Block-Co-Authors-Plus-Coauthors.Is-Layout-Flow, Class, Wp-Block-Co-Authors-Plus, Display Inline, .Wp-Block-Co-Authors-Plus-Avatar, Where Img, Height Auto Max-Width, Vertical-Align Bottom .Wp-Block-Co-Authors-Plus-Coauthors.Is-Layout-Flow .Wp-Block-Co-Authors-Plus-Avatar, Vertical-Align Middle .Wp-Block-Co-Authors-Plus-Avatar Is .Alignleft .Alignright
Content length: 1628 characters
Success: Alleged-rape-court-discharges-acquits-UNIBENs-lecturer-Ekundayo-three-years-after - 2024-10-26.docx
\n--- Processing article 3/101 ---
✅ Article saved as: Appeal-Court-acquits-pharmacist-jailed-for-life-over-rape - 2024-07-19.docx
Title: Appeal Court acquits pharmacist jailed for life over rape
Date: 2024-07-19
Authors: Eric Ikhilae, .Wp-Block-Co